# Analysis code for the paper "B1+ shimming in the cervical spinal cord at 7T"

## Data

The data can be downloaded at https://openneuro.org/datasets/ds004906

The structure of the input dataset is as follows (JSON sidecars are not listed for clarity):
~~~
ds004906
├── CHANGES
├── README
├── dataset_description.json
├── participants.json
├── participants.tsv
├── sub-01
│   ├── anat
│   │   ├── sub-01_acq-CP_T1w.nii.gz
│   │   ├── sub-01_acq-CP_T2starw.nii.gz
│   │   ├── sub-01_acq-CoV_T1w.nii.gz
│   │   ├── sub-01_acq-CoV_T2starw.nii.gz
│   │   ├── sub-01_acq-SAReff_T2starw.nii.gz
│   │   ├── sub-01_acq-patient_T2starw.nii.gz
│   │   ├── sub-01_acq-phase_T2starw.nii.gz
│   │   ├── sub-01_acq-target_T2starw.nii.gz
│   │   ├── sub-01_acq-volume_T2starw.nii.gz
│   └── fmap
│       ├── sub-01_acq-anatCP_TB1TFL.nii.gz
│       ├── sub-01_acq-anatCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-anatSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-anatpatient_TB1TFL.nii.gz
│       ├── sub-01_acq-anatphase_TB1TFL.nii.gz
│       ├── sub-01_acq-anattarget_TB1TFL.nii.gz
│       ├── sub-01_acq-anatvolume_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCP_TB1TFL.nii.gz
│       ├── sub-01_acq-fampCoV_TB1TFL.nii.gz
│       ├── sub-01_acq-fampSAReff_TB1TFL.nii.gz
│       ├── sub-01_acq-famppatient_TB1TFL.nii.gz
│       ├── sub-01_acq-fampphase_TB1TFL.nii.gz
│       ├── sub-01_acq-famptarget_TB1TFL.nii.gz
│       └── sub-01_acq-fampvolume_TB1TFL.nii.gz
├── sub-02
├── sub-03
├── sub-04
└── sub-05
~~~


## Overview of processing pipeline

For each subject:

- Segment the spinal cord on GRE scan
- Label vertebral levels on GRE scan using existing manual disc labels
- Extract the signal intensity on the GRE scan within the spinal cord
- Register each B1 map (CP, CoV, etc.) to the GRE scan
- Apply the computed warping field to bring the segmentation and vertebral levels to the B1 map
- Convert the B1 map to nT/V units
- Extract the B1 map value within the spinal cord

```{note}
Here is a note!
```
Slow processes are indicated with the emoji ⏳

In [ ]:
# Necessary imports

import os
import re
import json
import subprocess
import glob
import matplotlib.pyplot as plt
import numpy as np
import nibabel as nib
import pandas as pd
from scipy.interpolate import interp1d
from scipy.ndimage import uniform_filter1d

In [ ]:
# Download data and define path variables

!datalad install https://github.com/OpenNeuroDatasets/ds004906.git
os.chdir("ds004906")
!datalad get . # uncomment for production
# !datalad get sub-01/  # debugging
# Get derivatives containing manual labels
!datalad get derivatives

In [ ]:
# Define useful variables

path_data = os.getcwd()
print(f"path_data: {path_data}")
path_qc = os.path.join(path_data, "qc")
# shim_modes = ["CP", "CoV", "patient", "phase", "SAReff", "target", "volume"]
shim_modes = ["CP", "CoV"]  # debugging
print(f"shim_modes: {shim_modes}")
subjects = sorted(glob.glob("sub-*"))
print(f"subjects: {subjects}")

In [ ]:
# Run segmentation on GRE scan

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_deepseg_sc -i {subject}_acq-CoV_T2starw.nii.gz -c t2 -qc {path_qc}

## Verify QC report (GRE segmentation)

Open the quality control (QC) report located under `ds004906/qc/index.html`. Make sure the spinal cord segmentations are correct before resuming the analysis.

In [ ]:
# Crop GRE scan for faster processing and better registration results

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_crop_image -i {subject}_acq-CoV_T2starw.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop.nii.gz
    !sct_crop_image -i {subject}_acq-CoV_T2starw_seg.nii.gz -m {subject}_acq-CoV_T2starw_seg.nii.gz -dilate 20x20x0 -o {subject}_acq-CoV_T2starw_crop_seg.nii.gz

In [ ]:
# Label vertebrae on GRE scan

# Given the low resolution of the GRE scan, the automatic detection of C2-C3 disc is unreliable. Therefore we need to use the manual disc labels that are part of the dataset.
for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "anat"))
    fname_label_discs = os.path.join(path_data, "derivatives", "labels", subject, "anat", f"{subject}_acq-CoV_T2starw_label-discs_dseg.nii.gz")
    !sct_label_utils -i {subject}_acq-CoV_T2starw_crop_seg.nii.gz -disc {fname_label_discs} -o {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz
    # Generate QC report to assess labeled segmentation
    !sct_qc -i {subject}_acq-CoV_T2starw_crop.nii.gz -s {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -p sct_label_vertebrae -qc {path_qc} -qc-subject {subject}

In [ ]:
# Register the other shim methods to the GRE CoV scan

# TODO

In [ ]:
# Extract the signal intensity on the GRE scan within the spinal cord between levels C1 and T2 (included)

for subject in subjects:
    # TODO: loop across other shim methods    
    os.chdir(os.path.join(path_data, subject, "anat"))
    !sct_extract_metric -i {subject}_acq-CoV_T2starw_crop.nii.gz -f {subject}_acq-CoV_T2starw_crop_seg.nii.gz -method wa -vert 1:9 -vertfile {subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -append 1 -perslice 1 -o gre_CoV.csv


In [ ]:
# Register TFL flip angle maps to the GRE scan ⏳

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_register_multimodal -i {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -d ../anat/{subject}_acq-CoV_T2starw_crop.nii.gz -dseg ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -param step=1,type=im,algo=slicereg,metric=CC -qc {path_qc}

## Verify QC report (B1maps to GRE registration)

Open the QC report located under `ds004906/qc/index.html`. Make sure the registration are correct before resuming the analysis.

In [ ]:
# Warping spinal cord segmentation and vertebral level to each flip angle map

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_apply_transfo -i ../anat/{subject}_acq-CoV_T2starw_crop_seg.nii.gz -d {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -w warp_{subject}_acq-CoV_T2starw_crop2{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -x linear -o {subject}_acq-anat{shim_mode}_TB1TFL_seg.nii.gz
        !sct_apply_transfo -i ../anat/{subject}_acq-CoV_T2starw_crop_seg_labeled.nii.gz -d {subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -w warp_{subject}_acq-CoV_T2starw_crop2{subject}_acq-anat{shim_mode}_TB1TFL.nii.gz -x nn -o {subject}_acq-anat{shim_mode}_TB1TFL_seg_labeled.nii.gz

In [ ]:
# Convert the flip angle maps to B1+ efficiency maps [nT/V] (inspired by code from Kyle Gilbert)
# The approach consists in calculating the B1+ efficiency using a 1ms, pi-pulse at the acquisition voltage,
# then scale the efficiency by the ratio of the measured flip angle to the requested flip angle in the pulse sequence.

GAMMA = 2.675e8;  # [rad / (s T)]
requested_fa = 90  # saturation flip angle -- hard-coded in sequence

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        # Fetch the reference voltage from the JSON sidecar to the TFL B1map sequence
        with open(f"{subject}_acq-famp{shim_mode}_TB1TFL.json", "r") as f:
            metadata = json.load(f)
            ref_voltage = metadata.get("TxRefAmp", "N/A")
            print(f"ref_voltage [V]: {ref_voltage} ({subject}_acq-famp{shim_mode}_TB1TFL)")

        # Open flip angle map with nibabel
        nii = nib.load(f"{subject}_acq-famp{shim_mode}_TB1TFL.nii.gz")
        acquired_fa = nii.get_fdata()

        # Siemens maps are in units of flip angle * 10 (in degrees)
        acquired_fa = acquired_fa / 10

        # Account for the power loss between the coil and the socket. That number was given by Siemens.
        voltage_at_socket = ref_voltage * 10 ** -0.095

        # Compute B1 map in [T/V]
        b1_map = (acquired_fa / requested_fa) * (np.pi / (GAMMA * 1e-3 * voltage_at_socket))

        # Convert to [nT/V]
        b1_map = b1_map * 1e9

        # Save as NIfTI file
        nii_b1 = nib.Nifti1Image(b1_map, nii.affine, nii.header)
        nib.save(nii_b1, f"{subject}_acq-{shim_mode}_TB1map.nii.gz")

In [ ]:
# Extract B1+ value along the spinal cord between levels C1 and T2 (included)

for subject in subjects:
    os.chdir(os.path.join(path_data, subject, "fmap"))
    for shim_mode in shim_modes:
        !sct_extract_metric -i {subject}_acq-{shim_mode}_TB1map.nii.gz -f {subject}_acq-anat{shim_mode}_TB1TFL_seg.nii.gz -method wa -vert 1:9 -vertfile {subject}_acq-anat{shim_mode}_TB1TFL_seg_labeled.nii.gz -perslice 1 -o TB1map_{shim_mode}.csv

In [ ]:
# Make figure of B1+ values along the spinal cord across shim methods

def smooth_data(data, window_size=20):
    """ Apply a simple moving average to smooth the data. """
    return uniform_filter1d(data, size=window_size, mode='nearest')

# Fixed grid for x-axis
x_grid = np.linspace(0, 1, num_points)

# z-slices corresponding to levels C1 to T2 on the PAM50 template. These will be used to scale the x-label of each subject.
original_vector = np.array([984, 938, 907, 870, 833, 800, 769, 735, 692, 646])

# Normalize the PAM50 z-slice numbers to the 1-0 range (to show inferior-superior instead of superior-inferior)
min_val = original_vector.min()
max_val = original_vector.max()
normalized_vector = 1 - ((original_vector - min_val) / (max_val - min_val))

# Use this normalized vector as x-ticks
custom_xticks = normalized_vector

# Number of subjects determines the number of rows in the subplot
n_rows = len(subjects)

# Create a figure with multiple subplots
fig, axes = plt.subplots(n_rows, 1, figsize=(10, 6 * n_rows))

# Check if axes is an array or a single object
if n_rows == 1:
    axes = [axes]

# Iterate over each subject and create a subplot
for i, subject in enumerate(subjects):
    ax = axes[i]
    
    os.chdir(os.path.join(path_data, subject, "fmap"))

    for shim_method in shim_modes:
        # Initialize list to collect data for this shim method
        method_data = []

        for file_path in glob.glob(f"TB1map_*{shim_method}*.csv"):
            df = pd.read_csv(file_path)
            wa_data = df['WA()']

            # Normalize the x-axis to a 0-1 scale for each subject
            x_subject = np.linspace(0, 1, len(wa_data))

            # Interpolate to the fixed grid
            interp_func = interp1d(x_subject, wa_data, kind='linear', bounds_error=False, fill_value='extrapolate')
            resampled_data = interp_func(x_grid)

            # Apply smoothing
            smoothed_data = smooth_data(resampled_data)

            method_data.append(smoothed_data)

        # If there's data for this shim method, plot it
        if method_data:
            # Plotting each file's data separately
            for resampled_data in method_data:
                ax.plot(x_grid, resampled_data, label=f"{shim_method} - {file_path.split('/')[-1]}")

    # Set custom x-ticks
    ax.set_xticks(custom_xticks)
    ax.set_xticklabels([''] * len(original_vector))

    ax.set_title(f'{subject}')
#     ax.set_xlabel('Normalized Index')
    ax.set_ylabel('Values')

    # Add legend only to the first subplot
    if i == 0:
        ax.legend()

    ax.grid(True)

# Adjust the layout so labels and titles do not overlap
plt.tight_layout()
plt.show()